In [1]:
import os, shutil, itertools, yaml, kornia, torchvision
import dill as pickle
from importlib import reload
osp = os.path
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import torch
nn = torch.nn
F = nn.functional
import monai.transforms as mtr

In [2]:
import args as args_module
import jobs as job_mgmt
import optim, util, losses
from analysis import analyze
from data import dataloader
from models import model as model_module, common, img2d, ddpm, baseline
%matplotlib inline
plt.rcParams["figure.figsize"] = (4.0, 3.0)
plt.rcParams['figure.dpi'] = 200

rescale_clip = mtr.ScaleIntensityRangePercentiles(lower=1, upper=99, b_min=0, b_max=255, clip=True, dtype=np.uint8)
rescale_noclip = mtr.ScaleIntensityRangePercentiles(lower=0, upper=100, b_min=0, b_max=255, clip=False, dtype=np.uint8)
rescale_float = mtr.ScaleIntensity()

TMP_DIR=osp.expanduser("~/code/diffcoord/temp")

rsync -av --ignore-existing \
clintonw@peppercorn.csail.mit.edu:/data/vision/polina/users/clintonw/code/placenta/results/unetr_00 Downloads/

## interactive

In [ ]:
reload(dataloader);

In [3]:
args = job_mgmt.get_job_args("test")
args["data loading"]["dataset"] = "COCO"
data_loader = dataloader.get_dataloader(args)

In [8]:
reload(img2d);

In [42]:
param_dicts = torch.load(osp.join(TMP_DIR, "siren_63.pt"))
keys = ['net.0.linear.weight', 'net.0.linear.bias', 'net.1.linear.weight', 'net.1.linear.bias', 'net.2.linear.weight', 'net.2.linear.bias', 'net.3.linear.weight', 'net.3.linear.bias', 'net.4.weight', 'net.4.bias']
shape_by_key = {}
inputs = []
for k in keys:
    param_dicts[k] = torch.stack(param_dicts[k],0)
    shape_by_key[k] = param_dicts[k].shape[1:]
    inputs.append(param_dicts[k].flatten(start_dim=1))
inputs = torch.cat(inputs, dim=1)
inr_size = inputs.size(1)

In [ ]:
reload(baseline)

In [13]:
model = baseline.INR2INR(in_dim=inr_size, out_dim=inr_size, C=128).cuda()
diffusion = ddpm.GaussianDiffusion(
    model,
    timesteps = 1000,   # number of steps
    loss_type = 'l1',    # L1 or L2
    inr_size = inr_size,
).cuda()

In [14]:
loss = diffusion(inputs)
loss.backward()

In [ ]:
sampled_params = diffusion.sample(batch_size=4)

In [43]:
ix = 0
new_param_dict = {}
for k in keys:
    cur_shape = shape_by_key[k]
    nelem = int(np.prod(cur_shape))
    new_param_dict[k] = sampled_params[:, ix:ix+nelem].reshape(-1, *cur_shape)
    ix+=nelem
assert ix == sampled_params.size(1)

In [45]:
single_param_dict = {k:v[0] for k,v in new_param_dict.items()}

In [46]:
net = img2d.Siren().cuda()
net.load_state_dict(single_param_dict)

In [ ]:
reload(synthetic);

In [9]:
inputs[0].shape

torch.Size([63, 256, 2])

In [12]:
inputs[3].shape

torch.Size([63, 256])

In [8]:
len(inputs)

10

In [ ]:
## search python strings
import os
osp = os.path
root = osp.expanduser("~/code/placenta/placenta")
query_string = 'load_img'
for folder, subfolders, files in os.walk(root):
    for file in files:
        if file.endswith(".py"):
            path = osp.join(folder, file)
            with open(path, 'r') as f:
                if query_string in f.read():
                    print(path)